In [100]:
import pandas as pd
import numpy as np
import os
import sys
import geohash2 as geohash
from utils.df_preprocess import get_df_processed

df_obs_space = get_df_processed()

DEBUG:root:Column name: GEO1_Freq_Hz, Type: int64, Unique values: 1, Sample values: [1500000000]
DEBUG:root:Unique values: [1500000000]
DEBUG:root:Column name: LEO1_Freq_Hz, Type: int64, Unique values: 11, Sample values: [1500375000 1501875000]
DEBUG:root:Unique values: [0, 1498875000, 1500125000, 1500375000, 1500625000, 1500875000, 1501125000, 1501375000, 1501625000, 1501875000, 1502125000]
DEBUG:root:Column name: LEO2_Freq_Hz, Type: int64, Unique values: 11, Sample values: [1498875000 1500625000]
DEBUG:root:Unique values: [0, 1498875000, 1500125000, 1500375000, 1500625000, 1500875000, 1501125000, 1501375000, 1501625000, 1501875000, 1502125000]
DEBUG:root:Column name: LEO3_Freq_Hz, Type: int64, Unique values: 11, Sample values: [1502125000 1500125000]
DEBUG:root:Unique values: [0, 1498875000, 1500125000, 1500375000, 1500625000, 1500875000, 1501125000, 1501375000, 1501625000, 1501875000, 1502125000]
DEBUG:root:True
DEBUG:root:Column name: Time, Type: int64, Unique values: 182, Sample v

In [101]:
# # Encode into geohash
# leo1_geohash = geohash.encode(leo1_lat.item(), leo1_lon.item())

In [103]:
df_obs_space.columns

Index(['Time', 'GEO1_Melbourne_SNR_dB', 'GEO1_Melbourne_RSSI_dBm',
       'GEO1_Sydney_SNR_dB', 'GEO1_Sydney_RSSI_dBm', 'LEO1_Lat', 'LEO1_Lon',
       'LEO1_Melbourne_SNR_dB', 'LEO1_Melbourne_RSSI_dBm',
       'LEO1_Sydney_SNR_dB', 'LEO1_Sydney_RSSI_dBm', 'LEO2_Lat', 'LEO2_Lon',
       'LEO2_Melbourne_SNR_dB', 'LEO2_Melbourne_RSSI_dBm',
       'LEO2_Sydney_SNR_dB', 'LEO2_Sydney_RSSI_dBm', 'LEO3_Lat', 'LEO3_Lon',
       'LEO3_Melbourne_SNR_dB', 'LEO3_Melbourne_RSSI_dBm',
       'LEO3_Sydney_SNR_dB', 'LEO3_Sydney_RSSI_dBm'],
      dtype='object')

In [104]:
import pandas as pd
import numpy as np

def parse_satellite_data(row):
    leo_data = {}
    geo_data = {}

    for col in row.index:
        if col.startswith("LEO"):
            parts = col.split('_')
            sat_id = int(parts[0].replace("LEO", ""))
            field = parts[1]
            
            if field in ["Lat", "Lon"]:
                if sat_id not in leo_data:
                    leo_data[sat_id] = {'lat': None, 'lon': None, 'GS_metrics': {}}
                leo_data[sat_id][field.lower()] = row[col]
            else:
                gs_location = field
                metric = parts[2]
                if sat_id not in leo_data:
                    leo_data[sat_id] = {'lat': None, 'lon': None, 'GS_metrics': {}}
                if gs_location not in leo_data[sat_id]['GS_metrics']:
                    leo_data[sat_id]['GS_metrics'][gs_location] = {}
                leo_data[sat_id]['GS_metrics'][gs_location][metric] = row[col]

        elif col.startswith("GEO"):
            parts = col.split('_')
            sat_id = int(parts[0].replace("GEO", ""))
            gs_location = parts[1]
            metric = parts[2]

            if sat_id not in geo_data:
                geo_data[sat_id] = {'GS_metrics': {}}
            if gs_location not in geo_data[sat_id]['GS_metrics']:
                geo_data[sat_id]['GS_metrics'][gs_location] = {}
            geo_data[sat_id]['GS_metrics'][gs_location][metric] = row[col]

    return {'LEO': leo_data, 'GEO': geo_data}


In [106]:
parse_satellite_data(df_obs_space.iloc[100])

{'LEO': {1: {'lat': -3.65454458585465,
   'lon': 127.169756688252,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  2: {'lat': -3.66533965637565,
   'lon': 137.171129705337,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  3: {'lat': -3.67211466705473,
   'lon': 147.172603485687,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}}},
 'GEO': {1: {'GS_metrics': {'Melbourne': {'SNR': 47.0633078531765,
     'RSSI': -103.85688315986},
    'Sydney': {'SNR': 46.3783425398568, 'RSSI': -104.54184847318}}}}}

In [107]:
import pandas as pd
import re
import numpy as np

def build_satellite_data(df, row_index=0):
    result = {'LEO': {}, 'GEO': {}}

    for col in df.columns:
        if col == 'Time':
            continue

        match = re.match(r'(LEO|GEO)(\d+)_(.+)', col)
        if not match:
            continue

        sat_type, sat_num_str, rest = match.groups()
        sat_num = int(sat_num_str)

        if sat_type not in result:
            result[sat_type] = {}
        if sat_num not in result[sat_type]:
            result[sat_type][sat_num] = {}

        value = df.at[row_index, col] if col in df.columns else np.nan

        # Handle lat/lon
        if rest == 'Lat':
            result[sat_type][sat_num]['lat'] = value
        elif rest == 'Lon':
            result[sat_type][sat_num]['lon'] = value
        else:
            metric_match = re.match(r'(.+?)_(SNR|RSSI)', rest)
            if metric_match:
                location, metric = metric_match.groups()
                if 'GS_metrics' not in result[sat_type][sat_num]:
                    result[sat_type][sat_num]['GS_metrics'] = {}
                if location not in result[sat_type][sat_num]['GS_metrics']:
                    result[sat_type][sat_num]['GS_metrics'][location] = {'SNR': np.nan, 'RSSI': np.nan}
                result[sat_type][sat_num]['GS_metrics'][location][metric] = value

    return result


In [108]:
build_satellite_data(df_obs_space,0)

{'LEO': {1: {'lat': -65.2534969266155,
   'lon': 131.196415035991,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  2: {'lat': -65.2642338145129,
   'lon': 141.243932914994,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  3: {'lat': -65.2709744633676,
   'lon': 151.294798696656,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}}},
 'GEO': {1: {'GS_metrics': {'Melbourne': {'SNR': 47.0632141790182,
     'RSSI': -103.856976834019},
    'Sydney': {'SNR': 46.3782582120455, 'RSSI': -104.541932800991}}}}}

In [109]:
import pandas as pd
import re
import numpy as np

def build_all_satellite_data(df):
    results = []

    for i, row in df.iterrows():
        row_data = {'LEO': {}, 'GEO': {}}

        if 'Time' in df.columns:
            row_data['Time'] = row['Time']

        for col in df.columns:
            if col == 'Time':
                continue

            match = re.match(r'(LEO|GEO)(\d+)_(.+)', col)
            if not match:
                continue

            sat_type, sat_num_str, rest = match.groups()
            sat_num = int(sat_num_str)

            if sat_num not in row_data[sat_type]:
                row_data[sat_type][sat_num] = {}

            value = row[col]

            if rest == 'Lat':
                row_data[sat_type][sat_num]['lat'] = value
            elif rest == 'Lon':
                row_data[sat_type][sat_num]['lon'] = value
            else:
                metric_match = re.match(r'(.+?)_(SNR|RSSI)', rest)
                if metric_match:
                    location, metric = metric_match.groups()
                    if 'GS_metrics' not in row_data[sat_type][sat_num]:
                        row_data[sat_type][sat_num]['GS_metrics'] = {}
                    if location not in row_data[sat_type][sat_num]['GS_metrics']:
                        row_data[sat_type][sat_num]['GS_metrics'][location] = {'SNR': np.nan, 'RSSI': np.nan}
                    row_data[sat_type][sat_num]['GS_metrics'][location][metric] = value

        results.append(row_data)

    return results


In [116]:
build_all_satellite_data(df_obs_space.iloc[0:5])[0]

{'LEO': {1: {'lat': -65.2534969266155,
   'lon': 131.196415035991,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  2: {'lat': -65.2642338145129,
   'lon': 141.243932914994,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  3: {'lat': -65.2709744633676,
   'lon': 151.294798696656,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}}},
 'GEO': {1: {'GS_metrics': {'Melbourne': {'SNR': 47.0632141790182,
     'RSSI': -103.856976834019},
    'Sydney': {'SNR': 46.3782582120455, 'RSSI': -104.541932800991}}}},
 'Time': 1.7442864e+18}

In [ ]:
import numpy as np

# Get all column names
columns = df_obs_space.columns

# Find how many unique LEO users are in the DataFrame
leo_users = sorted(set(col.split('_')[0] for col in columns if col.startswith('LEO')))
geo_users = sorted(set(col.split('_')[0] for col in columns if col.startswith('GEO')))
# e.g., ['LEO1', 'LEO2', 'LEO3']


In [75]:
import numpy as np

# Dictionary to hold results
user_data = {
    'LEO': {},
    'GEO': {}
}

# Loop through all columns
for col in df_obs_space.columns:
    parts = col.split('_')
    if parts[0].startswith('LEO') or parts[0].startswith('GEO'):
        user_type = 'LEO' if parts[0].startswith('LEO') else 'GEO'
        user_id = parts[0]
        location = parts[1] if len(parts) > 2 else None
        metric = '_'.join(parts[2:]) if len(parts) > 2 else parts[1]

        # Initialize nested dict
        if user_id not in user_data[user_type]:
            user_data[user_type][user_id] = {}

        # Handle Lat/Lon separately
        if metric in ['Lat', 'Lon']:
            user_data[user_type][user_id][metric.lower()] = float(df_obs_space[col].iloc[0])
        else:
            # Handle RSSI and SNR (SINR)
            if location not in user_data[user_type][user_id]:
                user_data[user_type][user_id][location] = {}
            user_data[user_type][user_id][location][metric] = float(df_obs_space[col].iloc[0])


In [76]:
user_data

{'LEO': {'LEO1': {'lat': -65.2534969266155,
   'lon': 131.196415035991,
   'Melbourne': {'SNR_dB': nan, 'RSSI_dBm': nan},
   'Sydney': {'SNR_dB': nan, 'RSSI_dBm': nan}},
  'LEO2': {'lat': -65.2642338145129,
   'lon': 141.243932914994,
   'Melbourne': {'SNR_dB': nan, 'RSSI_dBm': nan},
   'Sydney': {'SNR_dB': nan, 'RSSI_dBm': nan}},
  'LEO3': {'lat': -65.2709744633676,
   'lon': 151.294798696656,
   'Melbourne': {'SNR_dB': nan, 'RSSI_dBm': nan},
   'Sydney': {'SNR_dB': nan, 'RSSI_dBm': nan}}},
 'GEO': {'GEO1': {'Melbourne': {'SNR_dB': 47.0632141790182,
    'RSSI_dBm': -103.856976834019},
   'Sydney': {'SNR_dB': 46.3782582120455, 'RSSI_dBm': -104.541932800991}}}}

In [77]:
leo1_lat = np.array(df_obs_space['LEO1_Lat'].iloc[0], dtype=np.float64)
leo1_lon = np.array(df_obs_space['LEO1_Lon'].iloc[0], dtype=np.float64)


In [83]:
df_initial = parse_satellite_data(df_obs_space.iloc[0])

In [84]:
df_initial

{'LEO': {1: {'lat': -65.2534969266155,
   'lon': 131.196415035991,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  2: {'lat': -65.2642338145129,
   'lon': 141.243932914994,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}},
  3: {'lat': -65.2709744633676,
   'lon': 151.294798696656,
   'GS_metrics': {'Melbourne': {'SNR': nan, 'RSSI': nan},
    'Sydney': {'SNR': nan, 'RSSI': nan}}}},
 'GEO': {1: {'GS_metrics': {'Melbourne': {'SNR': 47.0632141790182,
     'RSSI': -103.856976834019},
    'Sydney': {'SNR': 46.3782582120455, 'RSSI': -104.541932800991}}}}}

In [90]:
leo_data = df_initial['LEO']
# Extract lat/lon and encode using geohash
geohashes = [
    geohash.encode(entry['lat'], entry['lon']) 
    for entry in leo_data.values()
]

In [91]:
geohashes

['numkc2yqf492', 'phkkwp48q1jb', 'pk7sdvc9026x']

In [ ]:
# Assuming your dictionary is named df_initial
leo_data = df_initial['LEO']

# Extract lat/lon into a list of [lat, lon] pairs
lat_lon_list = [[entry['lat'], entry['lon']] for entry in leo_data.values()]

# Extract lat/lon pairs and geohash into a single list
lat_lon_geohash = [val for entry in leo_data.values() for val in (entry['lat'], entry['lon'])]

# Convert to NumPy array
lat_lon_array = np.array(lat_lon_list, dtype=np.float64)

# Flatten to 1D using np.ravel or .flatten
lat_lon_1d = np.ravel(lat_lon_array)  # or use lat_lon_2d.flatten()

print(lat_lon_1d)

[-65.25349693 131.19641504 -65.26423381 141.24393291 -65.27097446
 151.2947987 ]


In [98]:
df_initial = parse_satellite_data(df_obs_space.iloc[0])

# observation = {
#     "utc_time": np.array([df_obs_space['Time'].iloc[0]], dtype=np.int64),
#     "leo_pos": np.array([val for entry in leo_data.values() for val in (entry['lat'], entry['lon'])], dtype=np.float64),
#     "leo_rssi": np.array([val for entry in leo_data.values() for val in (entry['GS_metrics']['Melbourne']['RSSI_dBm'], entry['GS_metrics']['Sydney']['RSSI_dBm'])], dtype=np.float64),
#     "leo_snr": np.array([val for entry in leo_data.values() for val in (entry['GS_metrics']['Melbourne']['SNR_dB'], entry['GS_metrics']['Sydney']['SNR_dB'])], dtype=np.float64),
#     "geo_rssi": np.array([val for entry in df_initial['GEO'].values() for val in (entry['GS_metrics']['Melbourne']['RSSI_dBm'], entry['GS_metrics']['Sydney']['RSSI_dBm'])], dtype=np.float64),
#     "geo_snr": np.array([val for entry in df_initial['GEO'].values() for val in (entry['GS_metrics']['Melbourne']['SNR_dB'], entry['GS_metrics']['Sydney']['SNR_dB'])], dtype=np.float64),
# }


observation = {
    "utc_time": np.array([df_obs_space['Time'].iloc[0]], dtype=np.int64),
    "leo_pos": np.array([val for entry in leo_data.values() for val in (entry['lat'], entry['lon'])], dtype=np.float64),
    "leo_rssi": np.array([val for entry in leo_data.values() for val in (entry['GS_metrics']['Melbourne']['RSSI'], entry['GS_metrics']['Sydney']['RSSI'])], dtype=np.float64),
    "leo_snr": np.array([val for entry in leo_data.values() for val in (entry['GS_metrics']['Melbourne']['SNR'], entry['GS_metrics']['Sydney']['SNR'])], dtype=np.float64),
    "geo_rssi": np.array([val for entry in df_initial['GEO'].values() for val in (entry['GS_metrics']['Melbourne']['RSSI'], entry['GS_metrics']['Sydney']['RSSI'])], dtype=np.float64),
    "geo_snr": np.array([val for entry in df_initial['GEO'].values() for val in (entry['GS_metrics']['Melbourne']['SNR'], entry['GS_metrics']['Sydney']['SNR'])], dtype=np.float64),
}



In [99]:
observation

{'utc_time': array([1744286400000000000], dtype=int64),
 'leo_pos': array([-65.25349693, 131.19641504, -65.26423381, 141.24393291,
        -65.27097446, 151.2947987 ]),
 'leo_rssi': array([nan, nan, nan, nan, nan, nan]),
 'leo_snr': array([nan, nan, nan, nan, nan, nan]),
 'geo_rssi': array([-103.85697683, -104.5419328 ]),
 'geo_snr': array([47.06321418, 46.37825821])}

In [79]:
self.observation_space = Dict({
    "utc_time": Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.int64),
    "leo_pos": Box(low=-np.inf, high=np.inf, shape=(n_leo,), dtype=np.float64),
    "leo_rssi": Box(low=-np.inf, high=np.inf, shape=(n_leo*n_leo_users,), dtype=np.float64),
    "leo_sinr": Box(low=-np.inf, high=np.inf, shape=(n_leo*n_leo_users,), dtype=np.float64),
    "geo_rssi": Box(low=-np.inf, high=np.inf, shape=(n_geo*n_geo_users,), dtype=np.float64),            
    "geo_sinr": Box(low=-np.inf, high=np.inf, shape=(n_geo*n_geo_users,), dtype=np.float64),
})


NameError: name 'Dict' is not defined